In [187]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
#모델
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [110]:
data=pd.read_csv('train.csv')
real=pd.read_csv('test.csv')
submit=pd.read_csv('sample_submission.csv')

In [111]:
data.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [112]:
data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [113]:
data.groupby(['Transported','HomePlanet']).size()

Transported  HomePlanet
False        Earth         2651
             Europa         727
             Mars           839
True         Earth         1951
             Europa        1404
             Mars           920
dtype: int64

In [114]:
data['passenger']=data['PassengerId'].str[:4]

In [115]:
data['passenger']

0       0001
1       0002
2       0003
3       0003
4       0004
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: passenger, Length: 8693, dtype: object

In [116]:
home_passenger=data.groupby(['passenger','HomePlanet']).size().reset_index()
home_passenger

,passenger,HomePlanet,0
0,0001,Europa,1
1,0002,Earth,1
2,0003,Europa,2
3,0004,Earth,1
4,0005,Earth,1
...,...,...,...
6102,9275,Europa,3
6103,9276,Europa,1
6104,9278,Earth,1
6105,9279,Earth,1


In [117]:
home_passenger['passenger']

0       0001
1       0002
2       0003
3       0004
4       0005
        ... 
6102    9275
6103    9276
6104    9278
6105    9279
6106    9280
Name: passenger, Length: 6107, dtype: object

In [118]:
home_passenger['HomePlanet']

0       Europa
1        Earth
2       Europa
3        Earth
4        Earth
         ...  
6102    Europa
6103    Europa
6104     Earth
6105     Earth
6106    Europa
Name: HomePlanet, Length: 6107, dtype: object

In [119]:
home_dict={}
for i,j in zip(home_passenger['passenger'],home_passenger['HomePlanet']):
    home_dict[i]=j

In [120]:
home_dict

{'0001': 'Europa',
 '0002': 'Earth',
 '0003': 'Europa',
 '0004': 'Earth',
 '0005': 'Earth',
 '0006': 'Earth',
 '0007': 'Earth',
 '0008': 'Europa',
 '0009': 'Mars',
 '0010': 'Earth',
 '0011': 'Earth',
 '0012': 'Earth',
 '0014': 'Mars',
 '0015': 'Earth',
 '0016': 'Mars',
 '0017': 'Earth',
 '0020': 'Earth',
 '0022': 'Mars',
 '0024': 'Europa',
 '0025': 'Earth',
 '0026': 'Europa',
 '0028': 'Mars',
 '0030': 'Earth',
 '0031': 'Mars',
 '0034': 'Europa',
 '0035': 'Mars',
 '0036': 'Earth',
 '0038': 'Earth',
 '0039': 'Earth',
 '0041': 'Earth',
 '0043': 'Europa',
 '0044': 'Earth',
 '0045': 'Mars',
 '0050': 'Earth',
 '0051': 'Earth',
 '0052': 'Earth',
 '0053': 'Earth',
 '0056': 'Europa',
 '0058': 'Earth',
 '0061': 'Earth',
 '0062': 'Earth',
 '0064': 'Mars',
 '0066': 'Earth',
 '0067': 'Earth',
 '0068': 'Mars',
 '0069': 'Earth',
 '0070': 'Earth',
 '0071': 'Earth',
 '0072': 'Earth',
 '0073': 'Mars',
 '0074': 'Europa',
 '0076': 'Mars',
 '0077': 'Mars',
 '0078': 'Europa',
 '0081': 'Earth',
 '0082': 'Mar

In [121]:
data['passenger'].isnull().sum()

0

In [122]:
data['HomePlanet'].isnull().sum()

201

In [123]:
data['HomePlanet'].fillna(data['passenger'].map(home_dict),inplace=True)

In [124]:
data['HomePlanet'].isnull().sum()

111

In [125]:
data['HomePlanet'].isnull().sum()

111

In [126]:
data['Name2']=data.Name.str.split(' ').str[-1]
data['Name2']

0         Ofracculy
1             Vines
2            Susent
3            Susent
4       Santantines
           ...     
8688      Noxnuther
8689      Mondalley
8690         Connon
8691      Hontichre
8692      Hontichre
Name: Name2, Length: 8693, dtype: object

In [127]:
name_dict={}
for i, j in zip(data['Name2'],home_passenger['HomePlanet']):
    name_dict[i]=j

In [128]:
name_dict

{'Ofracculy': 'Europa',
 'Vines': 'Mars',
 'Susent': 'Earth',
 'Santantines': 'Earth',
 'Hinetthews': 'Earth',
 'Jacostaffey': 'Earth',
 'Beston': 'Mars',
 'Flatic': 'Earth',
 'Barne': 'Mars',
 'Baketton': 'Earth',
 'Bertsontry': 'Europa',
 'Pooles': 'Earth',
 'Eccle': 'Earth',
 'Hughriend': 'Europa',
 'Upead': 'Mars',
 'Brighttt': 'Earth',
 'Brantuarez': 'Earth',
 'Mcfaddennon': 'Earth',
 'Jacostanley': 'Europa',
 nan: 'Europa',
 'Fullided': 'Earth',
 'Brookenson': 'Earth',
 'Unconary': 'Earth',
 'Mare': 'Earth',
 'Morsentley': 'Mars',
 'Datie': 'Mars',
 'Oingwhedly': 'Earth',
 'Butte': 'Europa',
 'Leodger': 'Earth',
 'Wheelez': 'Earth',
 'Batthewitt': 'Earth',
 'Moodsey': 'Earth',
 'Cylistrand': 'Earth',
 'Coopelandez': 'Earth',
 'Chmad': 'Earth',
 'Lancis': 'Earth',
 'Johnshines': 'Earth',
 'Hubbarton': 'Earth',
 'Hickerson': 'Mars',
 'Tractive': 'Europa',
 'Ayalazquez': 'Earth',
 'Salez': 'Earth',
 'Greeves': 'Mars',
 'Keen': 'Mars',
 'Pecketton': 'Earth',
 'Leeves': 'Earth',
 'Bin

In [129]:
data['HomePlanet'].isnull().sum()

111

In [130]:
 5

5

In [131]:
data['HomePlanet'].isnull().sum()

111

In [132]:
data['HomePlanet'].value_counts(dropna=False)

Earth     4634
Europa    2161
Mars      1787
NaN        111
Name: HomePlanet, dtype: int64

In [133]:
data['HomePlanet'].fillna('Earth',inplace=True)

In [134]:
data['HomePlanet'].isnull().sum()

0

In [135]:
data['VIP'].fillna(False,inplace=True)

In [136]:
data['Fee']=(data['RoomService']+ data['FoodCourt']+ data['ShoppingMall']+ data['Spa']+ data['VRDeck'])

In [137]:
fee_list=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [138]:
for col in fee_list:
    data[col].fillna(data[col].median(),inplace=True)

In [139]:
data['RoomService'].isnull().sum()

0

In [140]:
data['Fee'].isnull().sum()

908

In [141]:
data['Cabin']

0          B/0/P
1          F/0/S
2          A/0/S
3          A/0/S
4          F/1/S
          ...   
8688      A/98/P
8689    G/1499/S
8690    G/1500/S
8691     E/608/S
8692     E/608/S
Name: Cabin, Length: 8693, dtype: object

In [142]:
data['deck']=data['Cabin'].str.split('/').str[0]

In [143]:
data['side']=data['Cabin'].str.split('/').str[2]

In [144]:
data.groupby(['HomePlanet','deck']).size().reset_index()

,HomePlanet,deck,0
0,Earth,A,1
1,Earth,B,1
2,Earth,C,4
3,Earth,D,4
4,Earth,E,411
5,Earth,F,1664
6,Earth,G,2559
7,Earth,T,1
8,Europa,A,255
9,Europa,B,778


In [145]:
data['deck'].fillna(data['HomePlanet'].map({'Earth':'G','Europa':'C','Mars':'F'}),inplace=True)

In [146]:
data['HomePlanet']=data['HomePlanet'].map({'Earth':'G','Europa':'C','Mars':'F'})

In [147]:
data.groupby(['passenger','side']).size().reset_index()

,passenger,side,0
0,0001,P,1
1,0002,S,1
2,0003,S,2
3,0004,S,1
4,0005,P,1
...,...,...,...
6113,9275,P,3
6114,9276,P,1
6115,9278,S,1
6116,9279,S,1


In [148]:
data['side'].fillna('S',inplace=True)

In [149]:
dest_passenger=data.groupby(['passenger','Destination']).size().reset_index()
dest_passenger

,passenger,Destination,0
0,0001,TRAPPIST-1e,1
1,0002,TRAPPIST-1e,1
2,0003,TRAPPIST-1e,2
3,0004,TRAPPIST-1e,1
4,0005,PSO J318.5-22,1
...,...,...,...
6875,9276,55 Cancri e,1
6876,9278,PSO J318.5-22,1
6877,9279,TRAPPIST-1e,1
6878,9280,55 Cancri e,1


In [150]:
dest_passenger['passenger']

0       0001
1       0002
2       0003
3       0004
4       0005
        ... 
6875    9276
6876    9278
6877    9279
6878    9280
6879    9280
Name: passenger, Length: 6880, dtype: object

In [151]:
dest_passenger['Destination']

0         TRAPPIST-1e
1         TRAPPIST-1e
2         TRAPPIST-1e
3         TRAPPIST-1e
4       PSO J318.5-22
            ...      
6875      55 Cancri e
6876    PSO J318.5-22
6877      TRAPPIST-1e
6878      55 Cancri e
6879      TRAPPIST-1e
Name: Destination, Length: 6880, dtype: object

In [152]:
dest_passenger=dest_passenger[dest_passenger['passenger'].duplicated()]

In [153]:
dest_passenger.sort_values(0,ascending=False)

,passenger,Destination,0
6513,8796,TRAPPIST-1e,7
4384,5885,TRAPPIST-1e,7
3175,4256,TRAPPIST-1e,7
156,0221,TRAPPIST-1e,6
6493,8770,TRAPPIST-1e,6
...,...,...,...
2774,3737,TRAPPIST-1e,1
2778,3742,TRAPPIST-1e,1
2786,3749,TRAPPIST-1e,1
2793,3755,TRAPPIST-1e,1


In [154]:
dest_passenger.drop_duplicates()

,passenger,Destination,0
8,0008,TRAPPIST-1e,1
17,0017,TRAPPIST-1e,1
19,0020,PSO J318.5-22,1
20,0020,TRAPPIST-1e,3
36,0044,PSO J318.5-22,1
...,...,...,...
6836,9219,TRAPPIST-1e,1
6838,9220,TRAPPIST-1e,2
6844,9227,TRAPPIST-1e,3
6847,9231,TRAPPIST-1e,2


In [155]:
dest_dict={}
for i,j in zip(dest_passenger['passenger'],dest_passenger['Destination']):
    dest_dict[i]=j

In [156]:
dest_dict

{'0008': 'TRAPPIST-1e',
 '0017': 'TRAPPIST-1e',
 '0020': 'TRAPPIST-1e',
 '0044': 'TRAPPIST-1e',
 '0067': 'TRAPPIST-1e',
 '0099': 'TRAPPIST-1e',
 '0103': 'TRAPPIST-1e',
 '0164': 'TRAPPIST-1e',
 '0220': 'TRAPPIST-1e',
 '0221': 'TRAPPIST-1e',
 '0244': 'TRAPPIST-1e',
 '0245': 'TRAPPIST-1e',
 '0257': 'TRAPPIST-1e',
 '0283': 'TRAPPIST-1e',
 '0285': 'TRAPPIST-1e',
 '0287': 'TRAPPIST-1e',
 '0290': 'TRAPPIST-1e',
 '0313': 'PSO J318.5-22',
 '0337': 'TRAPPIST-1e',
 '0358': 'TRAPPIST-1e',
 '0378': 'TRAPPIST-1e',
 '0383': 'TRAPPIST-1e',
 '0384': 'TRAPPIST-1e',
 '0417': 'TRAPPIST-1e',
 '0433': 'TRAPPIST-1e',
 '0436': 'PSO J318.5-22',
 '0437': 'TRAPPIST-1e',
 '0448': 'TRAPPIST-1e',
 '0453': 'TRAPPIST-1e',
 '0454': 'TRAPPIST-1e',
 '0457': 'TRAPPIST-1e',
 '0461': 'TRAPPIST-1e',
 '0470': 'TRAPPIST-1e',
 '0471': 'TRAPPIST-1e',
 '0476': 'TRAPPIST-1e',
 '0499': 'TRAPPIST-1e',
 '0504': 'TRAPPIST-1e',
 '0512': 'TRAPPIST-1e',
 '0527': 'TRAPPIST-1e',
 '0539': 'TRAPPIST-1e',
 '0549': 'TRAPPIST-1e',
 '0555': 'TR

In [157]:
data['Destination'].isna().sum()

182

In [158]:
data['Destination'].fillna(data['passenger'].map(dest_dict),inplace=True)

In [159]:
data['Destination'].isna().sum()

157

In [160]:
data['Destination'].sort_values(0,ascending=False)

0       TRAPPIST-1e
5246    TRAPPIST-1e
5265    TRAPPIST-1e
5261    TRAPPIST-1e
5260    TRAPPIST-1e
           ...     
8195            NaN
8217            NaN
8551            NaN
8616            NaN
8646            NaN
Name: Destination, Length: 8693, dtype: object

In [161]:
data['Destination'].fillna('TRAPPIST-1e',inplace=True)

In [162]:
data['CryoSleep'].value_counts()

False    5439
True     3037
Name: CryoSleep, dtype: int64

In [163]:
data['Fee']

0           0.0
1         736.0
2       10383.0
3        5176.0
4        1091.0
         ...   
8688     8536.0
8689        0.0
8690     1873.0
8691     4637.0
8692     4826.0
Name: Fee, Length: 8693, dtype: float64

In [164]:
data['CryoSleep']=np.where(data['Fee']>0,False,True)

In [165]:
data['Age'].fillna(data['Age'].median(),inplace=True)

In [166]:
data.isna().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           199
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
passenger         0
Name2           200
Fee             908
deck              0
side              0
dtype: int64

In [167]:
drop_list=['Name','Name2','Fee','Cabin','PassengerId']

In [168]:
data.drop(drop_list,axis=1,inplace=True)

In [169]:
data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,passenger,deck,side
0,C,True,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,B,P
1,G,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,F,S
2,C,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,A,S
3,C,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,A,S
4,G,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,F,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,C,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,9276,A,P
8689,G,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,9278,G,S
8690,G,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,9279,G,S
8691,C,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,9280,E,S


In [170]:
index=data.dtypes.index
value=data.dtypes.values

In [171]:
index

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported',
       'passenger', 'deck', 'side'],
      dtype='object')

In [172]:
obj_list=[]
for i,j in zip(index,value):
    if j=='object':
        obj_list.append(i)

In [173]:
obj_list

['HomePlanet', 'Destination', 'passenger', 'deck', 'side']

In [174]:
bool_list=[]
for i,j in zip(index,value):
    if j=='bool':
        bool_list.append(i)

In [175]:
bool_list

['CryoSleep', 'VIP', 'Transported']

In [176]:
for col in obj_list:
    label=LabelEncoder()
    data[col]=label.fit_transform(data[col])

In [177]:
for col in bool_list:
    label=LabelEncoder()
    data[col]=label.fit_transform(data[col])

In [178]:
data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,passenger,deck,side
0,0,1,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
1,2,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1,1,5,1
2,0,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,2,0,1
3,0,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,2,0,1
4,2,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,6213,0,0
8689,2,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,6214,6,1
8690,2,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,6215,6,1
8691,0,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,6216,4,1


In [179]:
X=data.drop('Transported',axis=1)
Y=data.Transported

In [180]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [181]:
model=RandomForestClassifier()

In [182]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [183]:
model.score(x_train,y_train)

0.9992809893586425

In [184]:
model.score(x_test,y_test)

0.7849338700402531

In [185]:
data.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
passenger       0
deck            0
side            0
dtype: int64

In [186]:
real

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [ ]:
s

In [190]:
pd.DataFrame(submit)

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False
